# Configs e Imports


In [ ]:
# instalar para o kaggle
%pip install sentence-transformers datasets accelerate

In [ ]:
%pip install -q --upgrade wandb

In [ ]:
# BIBLIOTECAS DE ANÁLISE DE DADOS E NUMÉRICAS
import numpy as np
import pandas as pd
from datasets import Dataset

# BIBLIOTECAS DE VISUALIZAÇÃO DE DADOS
import matplotlib.pyplot as plt
import seaborn as sns

# BIBLIOTECAS DE MACHINE LEARNING E ESTATÍSTICA
from scipy.stats import spearmanr
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

# BIBLIOTECA PRINCIPAL DE DEEP LEARNING (PYTORCH)
import torch

# BIBLIOTECA DE NLP (SENTENCE TRANSFORMERS)
from sentence_transformers import (
    InputExample,  # Estrutura para encapsular um exemplo de treino (par de frases + label).
    SentenceTransformer,  # Classe principal para carregar e usar modelos SBERT.
    losses,  # Módulo com as funções de perda (ex: CosineSimilarityLoss).
    util,  # Funções utilitárias (ex: util.cos_sim para similaridade).
)
from sentence_transformers.evaluation import (
    EmbeddingSimilarityEvaluator,
)  # Classe para avaliar a performance durante o treino.
from sentence_transformers.similarity_functions import (
    SimilarityFunction,
)  # Enum para funções de similaridade (ex: COSINE).
from sentence_transformers.trainer import (
    SentenceTransformerTrainer,
)  # API de alto nível para gerenciar o treinamento.
from sentence_transformers.training_args import (
    SentenceTransformerTrainingArguments,
)  # Argumentos e configurações para o Trainer.

# Define se o código rodará em GPU (cuda) ou CPU, otimizando a performance.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando device: {device}")

In [ ]:
from kaggle_secrets import UserSecretsClient
import wandb

user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")
wandb.login(key=wandb_api_key)

In [ ]:
# Configuração do WandB Sweep para Hyperparameter Tuning
sweep_config = {
    "method": "bayes",  # ou 'grid', 'random'
    "metric": {"name": "eval_validation_spearman_cosine", "goal": "maximize"},
    "parameters": {
        "learning_rate": {
            "distribution": "log_uniform_values",
            "min": 1e-6,
            "max": 5e-4,
        },
        "num_train_epochs": {"values": [2, 3, 4, 5, 6]},
        "per_device_train_batch_size": {"values": [8, 16, 32]},
        "warmup_ratio": {"distribution": "uniform", "min": 0.05, "max": 0.2},
        "weight_decay": {
            "distribution": "log_uniform_values",
            "min": 1e-6,
            "max": 1e-1,
        },
        "model_name": {
            "values": [
                "all-MiniLM-L6-v2",
                "all-mpnet-base-v2",
                "paraphrase-multilingual-MiniLM-L12-v2",
                "distilbert-base-multilingual-cased",
            ]
        },
    },
}

print("Configuração do sweep definida!")
print("Modelos disponíveis para teste:")
for model in sweep_config["parameters"]["model_name"]["values"]:
    print(f"  - {model}")

# EDA


In [ ]:
caminho_train = "/kaggle/input/2025-2-similaridade-de-sentencas/train.csv"
caminho_test = "/kaggle/input/2025-2-similaridade-de-sentencas/test.csv"

df = pd.read_csv(caminho_train)
print("Arquivo train.csv carregado com sucesso!")

# Exibir as primeiras linhas e informações básicas
print("\nDimensões do DataFrame:", df.shape)
print("\nColunas disponíveis:", df.columns.tolist())
print("\nPrimeiras 3 linhas do DataFrame:")
print(df.head(3))

print("\nValores nulos por coluna:")
print(df.isnull().sum())

# Análise de Balanceamento das Classes
print("\nDistribuição das classes:")
print(df["similarity_score"].value_counts().sort_index())

print("\nEstatísticas do similarity_score:")
print(df["similarity_score"].describe())

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.hist(df["similarity_score"], bins=20, alpha=0.7)
plt.title("Distribuição dos Scores de Similaridade")
plt.xlabel("Similarity Score")
plt.ylabel("Frequência")

plt.subplot(1, 2, 2)
sns.boxplot(y=df["similarity_score"])
plt.title("Boxplot dos Scores de Similaridade")
plt.show()

# Preparando Dados para o SBERT


In [ ]:
# Separar X e y
X = df[["sentence1", "sentence2"]]
y = df["similarity_score"]

# Separar dados em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar os scores para o intervalo [0, 1]
y_train_norm = y_train / 5.0
y_val_norm = y_val / 5.0

# Verificar os resultados
print("\nDimensões do conjunto de treino:", X_train.shape)
print("Dimensões do conjunto de validação:", X_val.shape)
print("\nDistribuição das classes no conjunto de treino:")
print(y_train.value_counts().sort_index())
print("\nDistribuição das classes no conjunto de validação:")
print(y_val.value_counts().sort_index())

# Preparar dados como Dataset do Hugging Face
train_dataset = Dataset.from_dict(
    {
        "sentence1": X_train["sentence1"].tolist(),
        "sentence2": X_train["sentence2"].tolist(),
        "label": y_train_norm.tolist(),
    }
)

val_dataset = Dataset.from_dict(
    {
        "sentence1": X_val["sentence1"].tolist(),
        "sentence2": X_val["sentence2"].tolist(),
        "label": y_val_norm.tolist(),
    }
)

# Preparando Dados para o SBERT
train_examples = [
    InputExample(texts=[s1, s2], label=float(label))
    for s1, s2, label in zip(X_train["sentence1"], X_train["sentence2"], y_train_norm)
]

val_examples = [
    InputExample(texts=[s1, s2], label=float(label))
    for s1, s2, label in zip(X_val["sentence1"], X_val["sentence2"], y_val)
]

print(f"\nNúmero de exemplos de treino: {len(train_examples)}")
print(f"Número de exemplos de validação: {len(val_examples)}")

# Configuração do Modelo SBERT


In [ ]:
# Função para configurar e treinar o modelo
def train_model_with_config(config=None):
    """
    Função para treinar o modelo com configurações específicas
    Pode ser usada tanto para treinamento único quanto para sweep
    """

    # Configurações padrão (caso não seja sweep)
    default_config = {
        "model_name": "all-MiniLM-L6-v2",
        "learning_rate": 2e-5,
        "num_train_epochs": 4,
        "per_device_train_batch_size": 16,
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
    }

    # Se config não for fornecido, usar padrões
    if config is None:
        config = default_config
    else:
        # Preencher valores faltantes com padrões
        for key, value in default_config.items():
            if key not in config:
                config[key] = value

    # Inicializar WandB
    run = wandb.init(
        project="sbert-similarity-optimization",
        config=config,
        tags=["sentence-similarity", "sbert", "fine-tuning"],
    )

    print("Iniciando treinamento com configurações:")
    for key, value in config.items():
        print(f"  {key}: {value}")

    # Configurar modelo
    model = SentenceTransformer(config["model_name"])
    model = model.to(device)

    # Configurar loss function
    train_loss = losses.CosineSimilarityLoss(model=model)

    # Configurar avaliador
    dev_evaluator = EmbeddingSimilarityEvaluator(
        sentences1=[example.texts[0] for example in val_examples],
        sentences2=[example.texts[1] for example in val_examples],
        scores=[example.label for example in val_examples],
        main_similarity=SimilarityFunction.COSINE,
        show_progress_bar=True,
        name="validation",
    )

    return model, train_loss, dev_evaluator, run, config


print("Função de configuração do modelo definida!")

# Treinamento do Modelo


In [ ]:
# Função para executar um experimento de treinamento
def train_experiment(config=None):
    """
    Executa um experimento completo de treinamento
    """

    # Configurar modelo e componentes
    model, train_loss, dev_evaluator, wandb_run, final_config = train_model_with_config(
        config
    )

    # Configurar argumentos de treinamento
    args = SentenceTransformerTrainingArguments(
        output_dir="./results",
        num_train_epochs=final_config["num_train_epochs"],
        per_device_train_batch_size=final_config["per_device_train_batch_size"],
        per_device_eval_batch_size=final_config["per_device_train_batch_size"],
        learning_rate=final_config["learning_rate"],
        warmup_ratio=final_config["warmup_ratio"],
        weight_decay=final_config["weight_decay"],
        fp16=True,
        dataloader_drop_last=False,
        eval_strategy="steps",
        eval_steps=50,
        save_strategy="steps",
        save_steps=50,
        save_total_limit=2,
        logging_steps=25,
        load_best_model_at_end=True,
        metric_for_best_model="eval_validation_spearman_cosine",
        greater_is_better=True,
        run_name=f"sbert-{final_config['model_name'].replace('/', '-')}-lr{final_config['learning_rate']:.1e}",
        report_to="wandb",  # Importante: enviar métricas para WandB
        remove_unused_columns=False,
    )

    # Configurar trainer
    trainer = SentenceTransformerTrainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        loss=train_loss,
        evaluator=dev_evaluator,
    )

    # Treinar o modelo
    trainer.train()

    # Avaliar no conjunto de validação
    val_score = dev_evaluator(model)
    main_score = val_score[dev_evaluator.primary_metric]

    # Registrar métricas adicionais no WandB
    wandb.log(
        {
            "final_validation_score": main_score,
            "model_name": final_config["model_name"],
            "total_parameters": sum(p.numel() for p in model.parameters()),
            "trainable_parameters": sum(
                p.numel() for p in model.parameters() if p.requires_grad
            ),
        }
    )

    print(f"Validation Score Final: {main_score:.4f}")

    # Finalizar WandB run
    wandb.finish()

    return model, main_score, final_config


print("Função de treinamento configurada!")

In [ ]:
# OPÇÃO 1: Treinamento único com configurações customizadas
def single_training():
    """Execute um único treinamento com configurações otimizadas"""

    # Configurações melhoradas baseadas em boas práticas
    optimized_config = {
        "model_name": "all-mpnet-base-v2",  # Modelo mais potente
        "learning_rate": 1e-5,  # Learning rate menor para fine-tuning
        "num_train_epochs": 3,
        "per_device_train_batch_size": 16,
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
    }

    print("=== EXECUTANDO TREINAMENTO ÚNICO ===")
    model, score, config = train_experiment(optimized_config)
    return model, score, config


# OPÇÃO 2: Hyperparameter Sweep
def run_hyperparameter_sweep(count=10):
    """Execute um sweep de hyperparâmetros"""

    print("=== EXECUTANDO HYPERPARAMETER SWEEP ===")
    print(f"Número de experimentos: {count}")

    # Inicializar sweep
    sweep_id = wandb.sweep(sweep_config, project="sbert-similarity-optimization")

    def sweep_train():
        # Esta função será chamada para cada experimento do sweep
        config = wandb.config
        train_experiment(dict(config))

    # Executar sweep
    wandb.agent(sweep_id, sweep_train, count=count)

    print("Sweep concluído! Verifique os resultados no WandB.")
    return sweep_id


print("Opções de treinamento configuradas!")
print("Execute uma das opções abaixo:")

In [ ]:
# EXECUTAR TREINAMENTO
# Escolha uma das opções abaixo:

# Opção 1: Treinamento único otimizado (mais rápido)
print("Iniciando treinamento único com configurações otimizadas...")
model, validation_score, final_config = single_training()

# Opção 2: Hyperparameter Sweep (mais demorado, mas pode encontrar melhores parâmetros)
# Descomente as linhas abaixo para executar o sweep:
# print("Iniciando hyperparameter sweep...")
# sweep_id = run_hyperparameter_sweep(count=5)  # Reduzido para 5 experimentos

print("\nTreinamento concluído!")
print(f"Score de validação: {validation_score:.4f}")
print("Configuração utilizada:")
for key, value in final_config.items():
    print(f"  {key}: {value}")

# Avaliação do Modelo


In [ ]:
# Avaliação Detalhada do Modelo Treinado

# Salvar o modelo treinado
caminho_modelo = "./fine-tuned-sbert-model"
model.save(caminho_modelo)
print("Modelo salvo!")

# Inicializar WandB para avaliação (se ainda não estiver ativo)
if not wandb.run:
    wandb.init(
        project="sbert-similarity-optimization",
        job_type="evaluation",
        tags=["evaluation", "final-model"],
    )

print("\nIniciando avaliação detalhada...")

# Gerar embeddings para as sentenças de validação
val_embeddings1 = model.encode(
    [example.texts[0] for example in val_examples],
    show_progress_bar=True,
    device=device,
    batch_size=32,
)
val_embeddings2 = model.encode(
    [example.texts[1] for example in val_examples],
    show_progress_bar=True,
    device=device,
    batch_size=32,
)

# Calcular similaridade coseno
similarities = util.cos_sim(val_embeddings1, val_embeddings2)
val_predictions = [similarities[i][i] for i in range(len(similarities))]

# Valores verdadeiros (escala original 0-5)
val_true = np.array([example.label for example in val_examples])
val_predictions_rescaled = np.array(val_predictions) * 5.0

# Métricas
mse = mean_squared_error(val_true, val_predictions_rescaled)
mae = mean_absolute_error(val_true, val_predictions_rescaled)
rmse = np.sqrt(mse)

# Correlações
pearson_corr = np.corrcoef(val_true, val_predictions_rescaled)[0, 1]
spearman_corr, _ = spearmanr(val_true, val_predictions_rescaled)

print("\n" + "=" * 50)
print("MÉTRICAS DE AVALIAÇÃO FINAL")
print("=" * 50)
print(f"MSE (0-5 scale): {mse:.4f}")
print(f"MAE (0-5 scale): {mae:.4f}")
print(f"RMSE (0-5 scale): {rmse:.4f}")
print(f"Pearson Correlation: {pearson_corr:.4f}")
print(f"Spearman Correlation: {spearman_corr:.4f}")
print("=" * 50)

# Registrar métricas no WandB
wandb.log(
    {
        "final_mse": mse,
        "final_mae": mae,
        "final_rmse": rmse,
        "final_pearson": pearson_corr,
        "final_spearman": spearman_corr,
        "validation_samples": len(val_true),
    }
)

# Análise de distribuição por classe
print("\nAnálise por classe de similaridade:")
for score in sorted(df["similarity_score"].unique()):
    mask = val_true == score
    if mask.sum() > 0:
        predicted_for_class = val_predictions_rescaled[mask]
        mean_pred = predicted_for_class.mean()
        std_pred = predicted_for_class.std()
        count = mask.sum()
        print(
            f"Classe {score}: {count:3d} amostras - Pred: {mean_pred:.3f}±{std_pred:.3f}"
        )

print(f"\nValidation Score Final: {spearman_corr:.4f}")

# Teste no Conjunto de Teste


In [ ]:
# Geração de Predições para o Conjunto de Teste

caminho_submissao = "./submission.csv"

print(f"Lendo dados de teste de '{caminho_test}'...")
df_test = pd.read_csv(caminho_test)

# Extrair as sentenças e os índices para o arquivo de submissão
test_sentences1 = df_test["sentence1"].tolist()
test_sentences2 = df_test["sentence2"].tolist()

# Verificar se existe coluna de índice explícita ou usar o índice do DataFrame
if "index" in df_test.columns:
    test_indexes = df_test["index"].tolist()
elif "id" in df_test.columns:
    test_indexes = df_test["id"].tolist()
else:
    test_indexes = df_test.index.tolist()

print(f"Encontrados {len(df_test)} exemplos no conjunto de teste.")
print("Primeiras 3 linhas do teste:")
print(df_test.head(3))

# Gerar Predições com otimizações
print("\nGerando embeddings e calculando similaridade...")
print("Utilizando batch processing otimizado...")

# Gerar embeddings com batch size otimizado
batch_size = 64  # Aumentado para maior eficiência
embeddings1 = model.encode(
    test_sentences1,
    show_progress_bar=True,
    device=device,
    batch_size=batch_size,
    normalize_embeddings=True,  # Normalizar para melhor performance do cosseno
)

embeddings2 = model.encode(
    test_sentences2,
    show_progress_bar=True,
    device=device,
    batch_size=batch_size,
    normalize_embeddings=True,
)

# Calcular similaridade de cosseno
test_similarities = util.cos_sim(embeddings1, embeddings2)

# As predições são a diagonal da matriz de similaridade (score de 0-1)
predictions_normalized = [
    test_similarities[i][i].item() for i in range(len(test_similarities))
]

# Reescalonar para 0-5 e aplicar clipping
predictions_rescaled = np.array(predictions_normalized) * 5.0
predictions_rescaled = np.clip(predictions_rescaled, 0, 5)

print(f"Predições geradas: {len(predictions_rescaled)}")
print(
    f"Range das predições: [{predictions_rescaled.min():.3f}, {predictions_rescaled.max():.3f}]"
)
print(f"Média das predições: {predictions_rescaled.mean():.3f}")

# Criar DataFrame de submissão
submission_df = pd.DataFrame(
    {"index": test_indexes, "predictions": predictions_rescaled}
)

# Salvar arquivo de submissão
submission_df.to_csv(caminho_submissao, index=False)
print(f"\nArquivo de submissão salvo em: {caminho_submissao}")

# Registrar estatísticas no WandB
if wandb.run:
    wandb.log(
        {
            "test_samples": len(predictions_rescaled),
            "test_pred_mean": predictions_rescaled.mean(),
            "test_pred_std": predictions_rescaled.std(),
            "test_pred_min": predictions_rescaled.min(),
            "test_pred_max": predictions_rescaled.max(),
        }
    )

# Verificação final
print("\nPrimeiras 10 linhas do arquivo de submissão:")
print(submission_df.head(10))

print("\nDistribuição das predições:")
print(submission_df["predictions"].describe())

In [ ]:
# Visualizar resultados
plt.figure(figsize=(18, 5))

# Gráfico 1: Scatter Plot (True vs Predicted na escala 0-1)
plt.subplot(1, 3, 1)
plt.scatter(val_true, val_predictions, alpha=0.6)
plt.plot([0, 1], [0, 1], "r--")
plt.xlabel("True Similarity")
plt.ylabel("Predicted Similarity")
plt.title("Validação: True vs Predicted (0-1)")
plt.grid(True)

# Gráfico 2: Histogramas de Distribuição (Validação, na escala 0-1)
plt.subplot(1, 3, 2)
plt.hist(val_predictions, bins=20, alpha=0.7, label="Predicted", density=True)
plt.hist(val_true, bins=20, alpha=0.7, label="True", density=True)
plt.xlabel("Similarity Score")
plt.ylabel("Frequency")
plt.title("Distribuição - Validação (0-1)")
plt.legend()

# Gráfico 3: Histograma de Distribuição (Teste, na escala 0-1)
plt.subplot(1, 3, 3)
plt.hist(predictions_normalized, bins=20, alpha=0.7, color="green")
plt.xlabel("Similarity Score")
plt.ylabel("Frequency")
plt.title("Distribuição - Teste (0-1)")

plt.tight_layout()
plt.show()

print("Análise completa!")

In [ ]:
# Análise Final e Recomendações para Melhoria

print("=" * 60)
print("RESUMO DO EXPERIMENTO E RECOMENDAÇÕES")
print("=" * 60)

# Estatísticas do modelo atual
print(f"Modelo utilizado: {final_config.get('model_name', 'N/A')}")
print(f"Score de validação: {validation_score:.4f}")
print(f"Configuração final: {final_config}")

print("\n" + "=" * 60)
print("ESTRATÉGIAS PARA MELHORAR O SCORE:")
print("=" * 60)

print("1. MODELOS ALTERNATIVOS para testar:")
print("   - sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
print("   - sentence-transformers/all-roberta-large-v1")
print("   - sentence-transformers/stsb-roberta-large")

print("\n2. TÉCNICAS DE ENSEMBLE:")
print("   - Combine predições de múltiplos modelos")
print("   - Use weighted averaging baseado na performance de validação")

print("\n3. OTIMIZAÇÕES DE TREINAMENTO:")
print("   - Aumente num_train_epochs para 5-8")
print("   - Teste learning_rates menores (5e-6, 1e-6)")
print("   - Use gradient accumulation para batch sizes maiores")

print("\n4. PRÉ-PROCESSAMENTO DOS DADOS:")
print("   - Limpeza mais agressiva de texto")
print("   - Normalização de caracteres especiais")
print("   - Remoção de stopwords específicas do domínio")

print("\n5. ANÁLISE DOS DADOS:")
print("   - Verifique se há overfitting comparando train vs validation")
print("   - Analise exemplos onde o modelo erra mais")
print("   - Considere data augmentation")

print("\n6. HYPERPARAMETER SWEEP COMPLETO:")
print("   - Execute sweep com mais experimentos (count=20-50)")
print("   - Teste diferentes combinações de modelo + learning rate")

if wandb.run:
    print("\n7. WANDB DASHBOARD:")
    print(f"   - Acesse: https://wandb.ai/{wandb.run.entity}/{wandb.run.project}")
    print("   - Compare diferentes experimentos")
    print("   - Analise a evolução das métricas durante o treinamento")

print("\n" + "=" * 60)
print("Para executar o hyperparameter sweep, descomente as linhas")
print("na célula de treinamento e execute novamente!")
print("=" * 60)